In [ ]:
# загрузка и предобработка
import pandas as pd 
import numpy as np
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

# https://github.com/huggingface/transformers
# обучение, загрузка по формату библиотеки, оптимизаторы
!pip install transformers
!pip install datasets
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
# обучение
import torch
from torch.utils.data import DataLoader
# предобработка и финальный отчет по метрикам
from sklearn import preprocessing
from sklearn.metrics import classification_report
# прогресс бар
from tqdm.auto import tqdm
# очистка кешей
import gc
# размер шрифта в графиках
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/pre_post_question_preparing.csv')

In [ ]:
import string
# реализуем предобработку
def preprocess(doc):
    # к нижнему регистру
    doc = doc.lower()
    # убираем пунктуацию, пробелы, прочее
    for p in string.punctuation + string.whitespace + 'http': 
        doc = doc.replace(p, ' ')
    # убираем кавычки
    for p in ['«', '»', '\'', '\"']:
        doc = doc.replace(p, ' ')
    # убираем лишние пробелы, объединяем обратно
    doc = doc.strip()
    doc = ' '.join([w for w in doc.split(' ') if w != ''])
    return doc

data['question'] = data['question'].map(preprocess)
data.head()

,question,label
0,are boo s cold reliable,0
1,is e main body rubber or las ic,0
2,ow long af er lacing order un il us delivery,0
3,will is fi a s ern mono oly mac ine,0
4,is e ower cord de ac able,0


In [ ]:
data.rename(columns={'label':'labels'}, inplace=True)
data

,question,labels
0,are boo s cold reliable,0
1,is e main body rubber or las ic,0
2,ow long af er lacing order un il us delivery,0
3,will is fi a s ern mono oly mac ine,0
4,is e ower cord de ac able,0
...,...,...
197669,w ose brig idea was i o fold is las ic collar ...,1
197670,if i seal e order i received i was e wrong rod...,1
197671,is is safe for dogs a swim in ond,1
197672,ese kee disconnec ing and reconnec ing o my on...,1


In [ ]:
df =  data.sample(frac=1, replace=False).reset_index(drop=True)
train = df[:int(len(df)*0.8)]
test =  df[int(len(df)*0.8):]

In [ ]:
len(train), len(test)

(158139, 39535)

In [ ]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

# Model&Trainig

In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [ ]:
# names = list(model.named_parameters())
# names[:6]

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cpu')

In [ ]:
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'test': 'test.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-93602c3c0d81fb33/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset = dataset.map(lambda e: tokenizer(e['question'], truncation=True, max_length =100, padding='max_length'), batched=True)

  0%|          | 0/159 [00:00<?, ?ba/s]

  0%|          | 0/40 [00:00<?, ?ba/s]

In [ ]:
torch_style_columns = ['input_ids', 'attention_mask', 'labels']
dataset = dataset.remove_columns(list(set(list(dataset['train'].features.keys())) - set(torch_style_columns))) 
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 158139
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 39535
    })
})

In [ ]:
dataset.set_format(type='torch', columns=torch_style_columns, device=device)

In [ ]:
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=1)
test_dataloader = DataLoader(dataset['test'], shuffle=True, batch_size=1)

In [ ]:
print(len(train_dataloader))
print(len(test_dataloader))

158139
39535


In [ ]:
# оптимизатор AdamW, лр стоит поперебирать
optimizer = AdamW(model.parameters(), lr=2e-6)
# количество эпох можно побольше поставить
num_epochs = 1
# количество шагов
num_training_steps = num_epochs * len(train_dataloader)

# будем линейно увеличивать первые 200 шагов
lr_scheduler = get_scheduler(
    "constant_with_warmup",
    optimizer=optimizer,
    num_warmup_steps=2000,
    num_training_steps=num_training_steps
)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
f1 = load_metric('f1')
acc = load_metric('accuracy')
precision = load_metric('precision')
recall = load_metric('recall')

<ipython-input-19-8b16a38d2f5c>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1 = load_metric('f1')


In [ ]:
def train(model, train_loader, test_loader, optimizer,lr_scheduler, num_epochs=num_epochs):
  best_f1 = 0 
  list_loss = []
  for _ in tqdm(range(num_epochs)):
    for i, batch in tqdm(enumerate(train_loader)):
      model.train()

      batch = {k:v.to(device)for k, v in batch.items()}
      output = model(**batch)
      optimizer.zero_grad()
      loss = output.loss
      list_loss.append(loss.item())
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      if i % 30000 == 0 and i!=0:
        with torch.no_grad():
          model.eval()
          for batch in tqdm(test_loader):
            batch = {k:v.to(device)for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            f1.add_batch(predictions=predictions, references=batch["labels"])    
            acc.add_batch(predictions=predictions, references=batch["labels"])    
            precision.add_batch(predictions=predictions, references=batch["labels"])    
            recall.add_batch(predictions=predictions, references=batch["labels"])

          print('weighted summary:')
          print('Test acc:', acc.compute()['accuracy'])
          print('Test precision:', precision.compute(average = 'weighted')['precision'])
          print('Test recall:', recall.compute(average = 'weighted')['recall'])
          f1_weighted = f1.compute(average = 'weighted')['f1']
          print('Test f1:', f1_weighted, '\n')
          # если текущая f1 лучше максимальной
          if f1_weighted > best_f1:
              # максмальная становится текущей
              best_f1 = f1_weighted
              # сохраняем модель
              model.save_pretrained(f'/content/drive/MyDrive/Colab Notebooks/hug_model_transformer_f1_{np.round(f1_weighted,5)}.pt')


In [ ]:
train(model, train_dataloader, test_dataloader, optimizer,lr_scheduler, num_epochs=num_epochs)

  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]